In [8]:
import requests
from bs4 import BeautifulSoup
import itertools 
from time import sleep
import numpy as np
  
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
  
# user define function
# Scrape the data
def getdata(url):
    r = requests.get(url, headers=HEADERS)
    return r.text
  

def html_code(url):
  
    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
  
    # display html code
    return (soup)
  
# key_word = 'laptop'
# url = f"https://www.amazon.com/s?k={key_word}&ref=nb_sb_noss"

#soup = html_code(url)
#print(soup)

In [ ]:
# key_word = 'laptop'
# page_num = 1
# max_page = 5
# url = f"https://www.amazon.com/s?k={key_word}&page={page_num}"
# soup = html_code(url)
# soup

In [5]:
def get_name_code(soup): #get laptop's name and product code
    # find the Html tag
    # with find()
    # and convert into string
    names = []
    codes = []
    for item in soup.find_all("a", {'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'}):
        #print(item)
        if item['href'][1:3] == 'gp':
            continue
        name = item['href'].split('/')[1]
        names.append(name)
        pdt_code = item['href'].split('/')[3]
        codes.append(pdt_code)
    return names, codes
  
#names, codes = get_name_code(soup)

In [6]:
def cus_rev(soup):
    data_str = []
    for item in soup.find_all("span", {'class':"a-size-base review-text review-text-content"}):
        data_str.append(item.get_text().strip())
    return (data_str)
  
#rev_data = cus_rev(soup)

In [9]:
def get_reviews_from_all_pages(key_word = 'laptop'):
    page_num = 1
    max_page = 5 #comment this for full result
    url = f"https://www.amazon.com/s?k={key_word}&page={page_num}"
    
    names = []
    codes = []
    #while True: #uncomment this for full result
    while page_num <= max_page: #comment this for full result
        soup = html_code(url)
        page_num += 1
        
        name, code = get_name_code(soup)
        if len(name) == 0:
            break
        names.extend(name)
        codes.extend(code)
        sleep(np.random.randint(2,20))
    print(len(names))
    reviews = []
    for name, code in zip(names, codes):
        url = f"https://www.amazon.com/{name}/product-reviews/{code}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
        soup = html_code(url)
        reviews.extend(cus_rev(soup))
        sleep(np.random.randint(2,20))
    
    return reviews

reviews = get_reviews_from_all_pages()

80


In [14]:
reviews[10]

"I was hesitant getting a Chromebook as I was concerned I'd miss being able to install windows programs, but so far I am really impressed with this. It was very quick to setup and easy to sync all of my Google account info. Having the ability to install Google apps is great. It's lightweight, runs fast, and seems to have a long lasting battery.As far as shipping it came packaged really well and the condition is 9/10 with only a slight smudge on the top of lid."

In [15]:
len(reviews)

565

In [17]:
import pandas as pd
df = pd.DataFrame({'reviews':reviews})
df.head()

,reviews
0,I could sit here and write all about the specs...
1,"This is the best laptop deal you can get, full..."
2,A very reasonably priced laptop for basic comp...
3,A few months after the purchase....It is still...
4,BUYER BE AWARE: This computer has Microsoft 10...


In [18]:
df.to_csv('laptop_reviews.csv')